# 🧹 Data Preprocessing & EDA
This notebook handles missing values, performs data cleaning, merges geolocation data, performs feature engineering, and prepares data for modeling.

### 📦 Imports

In [169]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import ipaddress


## 📂 Load Data

In [170]:
fraud_df = pd.read_csv('../../data/raw/Fraud_Data.csv')
ip_df = pd.read_csv('../../data/raw/IpAddress_to_Country.csv')
cc_df = pd.read_csv('../../data/raw/creditcard.csv')

In [171]:
fraud_df.sample(5)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
20191,164498,2015-06-10 12:43:52,2015-08-04 14:30:01,31,PTCQFALUMLIDM,SEO,Chrome,F,37,1.686584e+09,0
19991,77380,2015-05-22 11:27:50,2015-08-30 03:03:05,52,MSHLXSBYGGNLS,Ads,Chrome,M,40,9.802653e+08,0
20727,292040,2015-02-23 12:38:53,2015-05-18 18:32:37,29,DBNDLXYDAAQQV,SEO,Chrome,M,27,3.959296e+08,0
91024,229503,2015-03-12 16:54:41,2015-04-13 19:16:52,35,LTJOZAOQMFAUE,Ads,Chrome,F,40,3.137665e+09,0
58666,286946,2015-04-17 00:50:16,2015-07-02 19:41:35,11,LAYZLWKCEKKYN,Ads,FireFox,F,39,1.539085e+09,0


In [172]:
ip_df.sample(5)

,lower_bound_ip_address,upper_bound_ip_address,country
96598,3.331930e+09,3331930623,United States
121380,3.426674e+09,3426674431,United States
116470,3.407452e+09,3407452415,Australia
82835,3.256730e+09,3256731647,Finland
11368,1.168445e+09,1168449535,United States


In [173]:
cc_df.sample(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
171449,120642.0,2.092987,-0.830933,-1.494679,-0.896665,-0.290229,0.078276,-1.045032,0.137335,-0.142147,...,0.262008,0.719508,0.034197,-0.071294,-0.105338,-0.098544,0.016676,-0.017468,44.00,0
233812,147713.0,1.383996,-1.902264,-3.050818,-0.242012,-0.131733,-1.114415,0.993349,-0.578045,-1.249507,...,0.172327,0.038950,-0.540079,-0.429364,0.331477,0.986202,-0.218509,-0.031960,454.96,0
19703,30485.0,1.173396,-0.151056,0.426736,0.317258,-0.251219,0.318693,-0.308401,0.161866,0.403247,...,-0.032831,0.175401,-0.110795,-0.205570,0.495146,0.701685,-0.018501,-0.008521,9.00,0
209680,137660.0,0.967312,-1.617532,-1.953796,1.800170,-0.386636,-0.912500,1.109764,-0.419354,0.394325,...,0.229709,-0.469525,-0.317265,-0.052940,-0.055330,-0.745287,-0.113559,0.038766,549.56,0
258346,158610.0,1.948461,0.154752,-1.965084,1.311283,0.573014,-0.723803,0.257232,-0.189780,0.397287,...,0.197627,0.512757,-0.062225,0.348632,0.310104,-0.438251,-0.001436,-0.012616,56.50,0


## 🧼 Data Cleaning

### Missing Values

In [174]:
#count missing values in the df
fraud_df.isna().sum()

user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
dtype: int64

In [175]:
#check for missing values in the ip df
ip_df.isna().sum()

lower_bound_ip_address    0
upper_bound_ip_address    0
country                   0
dtype: int64

In [176]:
#check for missing values in credit card df
cc_df.isna().sum().sum()

0

In [177]:
# Convert timestamps
fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'])
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'])

# Remove duplicates
fraud_df.drop_duplicates(inplace=True)

In [178]:
fraud_df

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,XPSKTWGPWINLR,SEO,Chrome,M,28,3.451155e+09,1
151108,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,SEO,Safari,M,32,2.439047e+09,0
151109,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,SEO,IE,F,26,2.748471e+09,0
151110,207709,2015-07-09 20:06:07,2015-09-07 09:34:46,46,CMCXFGRHYSTVJ,SEO,Chrome,M,37,3.601175e+09,0


In [179]:
ip_df.dtypes

lower_bound_ip_address    float64
upper_bound_ip_address      int64
country                    object
dtype: object

## 🌐 IP Geolocation Mapping

In [180]:
fraud_df['ip_int'] = fraud_df['ip_address'].astype(float).astype(int)

# Make sure bounds are int
ip_df['lower_bound_ip_address'] = ip_df['lower_bound_ip_address'].astype(int)
ip_df['upper_bound_ip_address'] = ip_df['upper_bound_ip_address'].astype(int)

# Merge based on range: use pd.merge_asof or manual interval join
merged_df = pd.merge_asof(
    fraud_df.sort_values('ip_int'),
    ip_df.sort_values('lower_bound_ip_address'),
    left_on='ip_int',
    right_on='lower_bound_ip_address',
    direction='backward'
)

# Make sure bounds are int
merged_df['lower_bound_ip_address'] = merged_df['lower_bound_ip_address'].fillna(-1).astype('int')
merged_df['upper_bound_ip_address'] = merged_df['upper_bound_ip_address'].fillna(-1).astype('int')

# Optional: filter where ip_int is also <= upper_bound_ip_address
merged_df = merged_df[merged_df['ip_int'] <= merged_df['upper_bound_ip_address']]



In [181]:
# Check how many are missing
missing = merged_df['country'].isna().sum()
print(f"Missing countries: {missing}")

# Option 1: Fill with 'Unknown'
merged_df['country'].fillna('Unknown', inplace=True)

# Option 2: Drop if you're okay losing those rows
# fraud_df = fraud_df.dropna(subset=['country'])

Missing countries: 0


/tmp/ipykernel_340183/2064027834.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['country'].fillna('Unknown', inplace=True)


In [182]:
merged_df.sample(5)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,ip_int,lower_bound_ip_address,upper_bound_ip_address,country
96693,294185,2015-02-03 23:00:57,2015-05-20 13:05:21,29,TJZAXJVSTTDJX,Ads,IE,F,40,2.762070e+09,0,2762069521,2762014720,2762080255,Japan
102383,183504,2015-04-06 06:25:26,2015-04-11 00:51:51,15,IHQVNBVYYCVSV,Ads,IE,F,26,2.924205e+09,0,2924205161,2923429888,2924478463,United States
111088,320112,2015-04-11 15:17:37,2015-08-05 18:45:41,46,VOTAWRSTZVNIM,SEO,FireFox,M,25,3.181853e+09,0,3181852502,3181379584,3183476735,Mexico
21288,233454,2015-01-22 06:22:53,2015-02-03 07:01:41,33,FTDSHZBQBFSAF,SEO,Opera,F,33,5.857131e+08,0,585713096,570425344,587202559,United States
10252,261798,2015-01-12 06:25:49,2015-04-19 17:26:55,23,FCJGCGRPJDJAA,SEO,FireFox,M,33,2.800134e+08,0,280013364,268435456,285212671,United States


In [183]:
merged_df.isna().sum().sort_values(ascending=False)

user_id                   0
signup_time               0
purchase_time             0
purchase_value            0
device_id                 0
source                    0
browser                   0
sex                       0
age                       0
ip_address                0
class                     0
ip_int                    0
lower_bound_ip_address    0
upper_bound_ip_address    0
country                   0
dtype: int64

In [184]:
# Merge with country info
merged_df = pd.merge_asof(
    fraud_df.sort_values('ip_int'),
    ip_df.sort_values('lower_bound_ip_address'),
    left_on='ip_int', right_on='lower_bound_ip_address'
)
# Make sure bounds are int
merged_df['lower_bound_ip_address'] = merged_df['lower_bound_ip_address'].fillna(-1).astype('int')
merged_df['upper_bound_ip_address'] = merged_df['upper_bound_ip_address'].fillna(-1).astype('int')

In [185]:
merged_df.sample(5)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,ip_int,lower_bound_ip_address,upper_bound_ip_address,country
46845,326404,2015-04-07 10:16:05,2015-06-05 04:09:21,33,RMLYYRWCMZCEY,SEO,Chrome,M,33,1.332857e+09,0,1332857356,1332740096,1333264383,Romania
70691,121494,2015-01-01 09:14:58,2015-03-17 03:21:05,64,KAFWHSZKWADYZ,Ads,IE,F,36,2.021474e+09,0,2021473951,2019557376,2021654527,Taiwan; Republic of China (ROC)
126560,340375,2015-03-09 16:13:32,2015-04-25 07:46:46,72,VJYKBXGLRCKMO,Ads,Chrome,M,35,3.616816e+09,0,3616816498,3607101440,3623878655,United States
34124,173840,2015-01-29 01:56:37,2015-04-30 22:17:27,9,FLPTUCELCDMRF,Direct,IE,M,37,9.819128e+08,0,981912828,981467136,981991423,China
132928,244901,2015-05-13 03:18:07,2015-07-03 16:10:32,31,PEJPGUHWWDFQP,SEO,Chrome,M,33,3.791390e+09,0,3791389837,3758096128,3758096383,Australia


## 🧠 Feature Engineering

In [186]:
# Time since signup
merged_df['time_since_signup'] = (merged_df['purchase_time'] - merged_df['signup_time']).dt.total_seconds() / 3600
# Hour and day of transaction
merged_df['hour_of_day'] = merged_df['purchase_time'].dt.hour
merged_df['day_of_week'] = merged_df['purchase_time'].dt.dayofweek

In [187]:
merged_df.sample(5)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,ip_int,lower_bound_ip_address,upper_bound_ip_address,country,time_since_signup,hour_of_day,day_of_week
98539,364820,2015-04-28 08:15:58,2015-05-18 19:49:10,55,HUIBPKSKZUXPS,SEO,IE,F,28,2.812675e+09,0,2812674963,2812674048,2812739583,United States,491.553333,19,0
129416,222445,2015-04-28 00:50:10,2015-06-28 11:06:38,58,OSHBDEFXBBKKX,SEO,Safari,F,42,3.694574e+09,0,3694574327,3690987520,3695181823,Japan,1474.274444,11,6
61594,61478,2015-05-18 14:57:13,2015-07-17 16:49:21,49,OAMFCWKNZGCRC,SEO,FireFox,M,44,1.771114e+09,0,1771114123,1769996288,1772093439,Morocco,1441.868889,16,4
111720,21794,2015-03-23 04:36:01,2015-05-23 20:08:36,9,UWQHEEADIUBFG,Direct,Chrome,F,31,3.199499e+09,0,3199498519,3199467520,3199500287,Bolivia,1479.543056,20,5
16865,347642,2015-04-24 22:58:00,2015-05-22 00:06:52,21,KYZOQXVPDFHUW,SEO,Chrome,M,32,4.634825e+08,0,463482521,463470592,464519167,Korea Republic of,649.147778,0,4


In [188]:
merged_df.rename(columns={
    'class': 'is_fraud',
    'ip_address': 'ip_address_raw'
}, inplace=True)


### Rename columns 

In [189]:
merged_df.sample()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address_raw,is_fraud,ip_int,lower_bound_ip_address,upper_bound_ip_address,country,time_since_signup,hour_of_day,day_of_week
15975,50444,2015-01-09 23:38:55,2015-02-01 16:35:00,67,LLAZKHGJIDPJK,Ads,FireFox,F,28,4.405427e+08,0,440542678,436207616,452984831,United States,544.934722,16,6


### Reorder important columns

In [190]:
cols = [
    'user_id', 'signup_time', 'purchase_time', 'time_since_signup',
    'purchase_value', 'device_id', 'source', 'browser', 'sex', 'age',
    'ip_address_raw', 'ip_int', 'country', 'hour_of_day', 'day_of_week',
    'is_fraud'
]
final_df = merged_df[cols]


In [191]:
# Transaction frequency and velocity features
fraud_df.groupby('user_id')['purchase_time'].count()
fraud_df.groupby('device_id')['purchase_time'].nunique().sort_values(ascending=False)

device_id
KIPFSCNUGOLDP    20
NGQCKIADMZORL    20
CQTUVBYIWWWBC    20
ITUMJCKWEYNDD    20
EQYVNEGOFLAWK    20
                 ..
IXVBQLPWSAIDA     1
IXUYCZZVDXGPO     1
IXUWTGATQJEVG     1
IXUKDXQBVCYCZ     1
ZZZXASJUVUNMV     1
Name: purchase_time, Length: 137956, dtype: int64

In [192]:
#Handle Class Imbalance
fraud_df['class'].value_counts(normalize=True)

class
0    0.906354
1    0.093646
Name: proportion, dtype: float64

In [193]:
final_df.sample(1)

,user_id,signup_time,purchase_time,time_since_signup,purchase_value,device_id,source,browser,sex,age,ip_address_raw,ip_int,country,hour_of_day,day_of_week,is_fraud
77844,33596,2015-08-16 22:46:13,2015-08-23 16:12:56,161.445278,13,FYJCACXBLXOCF,SEO,Chrome,M,32,2.218369e+09,2218369465,United States,16,6,0


### Normalization and Scaling

In [194]:
from sklearn.preprocessing import StandardScaler

# Choose numeric features to scale
numeric_features = ['purchase_value', 'age', 'time_since_signup', 'hour_of_day', 'day_of_week']

scaler = StandardScaler()
final_df[numeric_features] = scaler.fit_transform(final_df[numeric_features])

# Optional: view result
print(final_df[numeric_features].head())


   purchase_value       age  time_since_signup  hour_of_day  day_of_week
0        0.494721  0.331793          -1.013679    -0.220124     1.489476
1       -0.214781 -0.364448          -1.230613     0.792542     0.492565
2       -0.214781 -0.132367          -1.337931    -0.509456    -1.002803
3       -0.214781  0.795954           0.800513     1.371208    -0.005891
4        0.985915  0.563874          -1.127359    -0.654123     1.489476


/tmp/ipykernel_340183/1117632796.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[numeric_features] = scaler.fit_transform(final_df[numeric_features])


### Encoding Categorical Features

In [195]:
# Use pandas get_dummies for simplicity
categorical_features = ['browser', 'source', 'sex']

final_df_encoded = pd.get_dummies(final_df, columns=categorical_features, drop_first=True)

# View result
print(final_df_encoded.columns)


Index(['user_id', 'signup_time', 'purchase_time', 'time_since_signup',
       'purchase_value', 'device_id', 'age', 'ip_address_raw', 'ip_int',
       'country', 'hour_of_day', 'day_of_week', 'is_fraud', 'browser_FireFox',
       'browser_IE', 'browser_Opera', 'browser_Safari', 'source_Direct',
       'source_SEO', 'sex_M'],
      dtype='object')


In [196]:
final_df_encoded.sample(5)

,user_id,signup_time,purchase_time,time_since_signup,purchase_value,device_id,age,ip_address_raw,ip_int,country,hour_of_day,day_of_week,is_fraud,browser_FireFox,browser_IE,browser_Opera,browser_Safari,source_Direct,source_SEO,sex_M
37882,96835,2015-05-21 02:50:20,2015-09-10 09:37:31,1.525533,1.531686,LOXYMZSEXDSFR,-0.480488,1.089267e+09,1089266908,United States,-0.364790,-0.005891,1,False,True,False,False,False,True,True
99536,32065,2015-02-28 05:12:37,2015-05-04 07:02:24,0.220890,1.095069,FDSPQKYYPZXOY,-0.712568,2.842713e+09,2842712609,United States,-0.654123,-1.501259,0,False,False,False,False,False,False,True
121876,139687,2015-01-02 07:54:51,2015-01-14 19:53:58,-1.232172,-0.433090,RMQELETYNELAX,-1.292769,3.488550e+09,3488549814,United States,1.081875,-0.504347,0,False,False,False,False,False,True,True
22978,370968,2015-05-19 17:13:24,2015-07-16 02:30:31,0.008380,-0.542244,XKFBSVEDBDGNZ,-0.132367,6.319703e+08,631970271,France,-1.377455,-0.005891,0,True,False,False,False,False,True,True
82207,299996,2015-03-08 08:30:49,2015-03-25 16:52:49,-1.098155,-0.978861,JAFFGWGOEPWVV,0.447833,2.339508e+09,2339507645,Norway,0.647875,-0.504347,0,False,False,False,False,True,False,True


### Save to CSV for modeling:

In [197]:
final_df.to_csv("../../data/interim/interim_fraud_data.csv", index=False)

## ⚖️ Handle Class Imbalance (SMOTE)

In [198]:
features = merged_df[['purchase_value', 'time_since_signup', 'hour_of_day', 'day_of_week']]
target = merged_df['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, stratify=target)
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [199]:
X_resampled.shape

(191744, 4)